CNN卷积神经网络处理MNIST

In [10]:
#定义CNN
import torch
import torch.nn as nn
import torch.nn.functional as F

class LeNet(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet, self).__init__()
        # 定义卷积层
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2)
        #上方输入1通道，输出6通道 1*28*28 -> 6
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5, stride=1)
        #上方输入6通道，输出16通道 6 -> 16
        #conv2d函数详解
        #torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
        #in_channels：输入通道数
        #out_channels：输出通道数
        #kernel_size：卷积核大小,如果是整数，则表示卷积核的高和宽相同；如果是元组，则表示卷积核的高和宽分别为元组的两个元素
        #stride：步长，默认为1，整数则代表步长相同；元组则代表高和宽的步长分别为元组的两个元素
        #padding：填充，默认为0

        #定义全连接层
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        #上方输入16*5*5通道，输出120通道
        self.fc2 = nn.Linear(120, 84)
        #上方输入120通道，输出84通道
        self.fc3 = nn.Linear(84, num_classes)
        #上方输入84通道，输出num_classes通道

    # 定义前向传播
    def forward(self, x):
        # 卷积层1+激活+池化
        x = F.relu(self.conv1(x))
        x = F.avg_pool2d(x, kernel_size=2)
        # 卷积层2+激活+池化
        x = F.relu(self.conv2(x))
        x = F.avg_pool2d(x, kernel_size=2)
        # 展平
        x = torch.flatten(x,start_dim=1)  #将16*5*5的特征图展平为一维向量
        # 全连接层1+激活
        x = F.relu(self.fc1(x))
        # 全连接层2+激活
        x = F.relu(self.fc2(x))
        # 全连接层3
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)  # 使用log_softmax作为输出激活函数
        

训练与测试函数（同SimpleNN）

In [11]:
#定义训练函数
def train(args, model, device, train_loader, optimizer, epoch):
# args:一般是一个包含超参数的对象或命名空间（如batch_size, epochs等），用于控制训练过程
# model: 定义的神经网络模型
# device: 设备（CPU或GPU），用于将数据和模型移动到相应的设备上
# train_loader: 数据加载器，用于加载训练数据，包含训练数据的批次，用于迭代访问每个batch
# optimizer: 优化器，用于更新模型参数（如SGD, Adam等）
# epoch: 当前训练的轮数，用于记录训练进度
    model.train()  # 设置模型为训练模式，pytorch有train和eval两种模式，train模式会启用dropout和batch normalization等训练时特有的操作
    for batch_idx, (data, target) in enumerate(train_loader):#batch_idx是批次索引，data是输入数据，target是目标标签
        data, target = data.to(device), target.to(device)  # 将数据和目标移动到设备上
        optimizer.zero_grad()  # 清除梯度
        output = model(data)  # 前向传播,output是模型的输出(预测结果)
        loss = F.nll_loss(output, target)  # 计算损失(预测值与真实标签的差值),F.nll_loss是负对数似然损失函数，适用于多分类问题
        loss.backward()  # loss反向传播
        optimizer.step()  # l利用计算得到参数更新梯度

#定义测试函数
def test(model, device, test_loader):
    #model: 定义的神经网络模型
    #device: 设备（CPU或GPU），用于将数据和模型移动到相应的设备上
    #test_loader: 数据加载器，用于加载测试数据，包含测试数据的批次，用于迭代访问每个batch
    model.eval()  # 设置模型为评估模式，pytorch有train和eval两种模式，eval模式会禁用dropout和batch normalization等训练时特有的操作
    test_loss = 0
    correct = 0
    with torch.no_grad():  # 在测试时不需要计算梯度
        for data, target in test_loader:  # 遍历测试数据
            data, target = data.to(device), target.to(device)  # 将数据和目标移动到设备上
            output = model(data)  # 前向传播
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # 计算损失，reduction='sum'表示对所有样本的损失
            pred = output.argmax(dim=1, keepdim=True)  # 获取预测结果，pred是一个张量，包含每个样本的预测类别
            correct += pred.eq(target.view_as(pred)).sum().item()  # 计算正确预测的数量，target.view_as(pred)将target的形状调整为与pred相同，便于比较
    test_loss /= len(test_loader.dataset)  # 计算平均损失

    print('='* 50)
    print("Evaluation Results:")
    print(f"Average loss: {test_loss:.4f}")
    print(f"Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.2f}%)")
    print('='* 50)         

In [12]:
#模型运行
epochs = 15 # 训练轮数
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 检查是否有可用的GPU
print(device)

cuda


In [13]:
import torch.optim as optim
# 定义优化器
model = LeNet().to(device)  # 实例化模型并移动到设备上
optimizer = optim.Adadelta(model.parameters())  # 使用Adadelta优化器


In [14]:
from torchvision import datasets, transforms
# 定义数据转换
transform = transforms.Compose([
    transforms.ToTensor(),  # 将图像转换为Tensor
    transforms.Normalize((0.1307,), (0.3081,))  # 对图像进行归一化处理
])  

dataset1 = datasets.MNIST(root='./data', download=True, train=True,  transform=transform)  # 训练集
dataset2 = datasets.MNIST(root='./data', train=False,  download=True, transform=transform)  # 测试集
train_loader = torch.utils.data.DataLoader(dataset1, batch_size=64, shuffle=True)  # 训练数据加载器
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=1000, shuffle=False)  # 测试数据加载器

100%|██████████| 28.9k/28.9k [00:00<00:00, 115kB/s]
100%|██████████| 1.65M/1.65M [00:07<00:00, 228kB/s] 
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.18MB/s]


In [15]:
dataset1

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [ ]:
import time
for epoch in range(1, epochs + 1):
    start_time = time.time() #记录开始时间
    train([],model,device,train_loader,optimizer,epoch)  # 训练模型
    train_time = time.time()
    test(model, device, test_loader)  # 测试模型
    test_time = time.time()
    train_duration = train_time - start_time
    test_duration = test_time - train_time
    print(f"Epoch {epoch} completed in {train_duration:.2f} seconds for training and {test_duration:.2f} seconds for testing.")
# --- END OF RECENT EDITS ---
# --- IGNORE ---
# --- END OF IGNORE ---

Evaluation Results:
Average loss: 0.0628
Accuracy: 9808/10000 (98.08%)
Epoch 1 completed in 8.10 seconds for training and 0.97 seconds for testing.
Evaluation Results:
Average loss: 0.0539
Accuracy: 9822/10000 (98.22%)
Epoch 2 completed in 7.48 seconds for training and 0.90 seconds for testing.
Evaluation Results:
Average loss: 0.0362
Accuracy: 9903/10000 (99.03%)
Epoch 3 completed in 7.53 seconds for training and 0.90 seconds for testing.
Evaluation Results:
Average loss: 0.0496
Accuracy: 9848/10000 (98.48%)
Epoch 4 completed in 7.52 seconds for training and 0.91 seconds for testing.
Evaluation Results:
Average loss: 0.0354
Accuracy: 9893/10000 (98.93%)
Epoch 5 completed in 7.29 seconds for training and 0.91 seconds for testing.
Evaluation Results:
Average loss: 0.0408
Accuracy: 9899/10000 (98.99%)
Epoch 6 completed in 7.38 seconds for training and 0.90 seconds for testing.
Evaluation Results:
Average loss: 0.0347
Accuracy: 9905/10000 (99.05%)
Epoch 7 completed in 7.50 seconds for tra